In [67]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [49]:
def lagit(df, langs):
  for i in range(1, langs+1):
    df['Lang_'+str(i)] = df['ret'].shift(i)
  return ['Lang_'+str(i) for i in range(1, langs+1)]

In [48]:
start_date = '2010-01-01'
end_date =  '2023-02-08'
df= pd.read_csv("E:\\Projetos\\BotTrading-Quantil\\BotTrading\\notebooks\\DOL1.csv", parse_dates=['time'])
df = df.loc[(df['time'] >= start_date) & (df['time'] <= end_date)]
df.head()

,time,open,high,low,close,Volume,Volume MA
1753,2010-01-04 08:00:00,1740.0,1743.5,1727.5,1730.651,177690.0,177690.0
1754,2010-01-05 08:00:00,1729.0,1747.0,1725.5,1737.951,222370.0,222370.0
1755,2010-01-06 08:00:00,1739.0,1746.5,1735.0,1740.271,226790.0,226790.0
1756,2010-01-07 08:00:00,1746.0,1756.5,1742.5,1748.761,289000.0,289000.0
1757,2010-01-08 08:00:00,1754.0,1758.5,1733.0,1736.495,269765.0,269765.0


**Manipulação para o modelo e predição do retorno dos dias anteriores**

In [50]:
predicotors = ['open' , 'high', 'low', 'close', 'Volume', 'Volume MA']
df['direction'] = (df['close'] > df['close'].shift(1)).astype(int)

In [51]:
df.tail()

,time,open,high,low,close,Volume,Volume MA,direction
4993,2023-02-01 09:00:00,5091.0,5135.5,5065.5,5112.550,228800.0,228800.0,1
4994,2023-02-02 09:00:00,5050.0,5078.5,4963.5,5056.572,334135.0,334135.0,0
4995,2023-02-03 09:00:00,5120.0,5180.0,5079.5,5153.317,296025.0,296025.0,1
4996,2023-02-06 09:00:00,5190.0,5233.0,5160.0,5202.935,256735.0,256735.0,1
4997,2023-02-07 09:00:00,5163.0,5237.0,5151.5,5218.006,280820.0,280820.0,1


In [52]:
df['ret'] = df.close.pct_change()
lagit(df, 2)
df['direction'] = np.where(df.ret > 0, 1, 0)

In [53]:
df.tail()

,time,open,high,low,close,Volume,Volume MA,direction,ret,Lang_1,Lang_2
4993,2023-02-01 09:00:00,5091.0,5135.5,5065.5,5112.550,228800.0,228800.0,1,0.001629,-0.002172,0.001330
4994,2023-02-02 09:00:00,5050.0,5078.5,4963.5,5056.572,334135.0,334135.0,0,-0.010949,0.001629,-0.002172
4995,2023-02-03 09:00:00,5120.0,5180.0,5079.5,5153.317,296025.0,296025.0,1,0.019133,-0.010949,0.001629
4996,2023-02-06 09:00:00,5190.0,5233.0,5160.0,5202.935,256735.0,256735.0,1,0.009628,0.019133,-0.010949
4997,2023-02-07 09:00:00,5163.0,5237.0,5151.5,5218.006,280820.0,280820.0,1,0.002897,0.009628,0.019133


In [55]:
features = lagit(df, 3)

In [58]:
df.dropna(inplace=True)

In [61]:
X = df[features]
y = df['direction']

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.3)

In [70]:
from keras.models import Sequential
from keras.layers import Dense


In [ ]:
model = Sequential()
model.add(Dense(units=50, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=25, activation='relu'))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)


In [82]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Score:", score)


Score: [0.6927942633628845, 0.5354573726654053]
